In [15]:
#import modules

import numpy as np
import os
import matplotlib.pyplot as plt
import pickle

from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.utils import multi_gpu_model
from keras.utils import Sequence
from keras.models import load_model
from tensorflow.distribute import MirroredStrategy

#initialise random generator
rng = np.random.default_rng()

In [16]:
#Define FelixDataflow classes and functions.

class FelixSequence(Sequence):
    def __init__(self, x_set, y_set, batch_size, file_type):
        """Here self.x is a list of paths to file_type files. self.y is a
        corresponding list of labels."""
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.file_type = file_type

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return arrs_from_paths(batch_x, self.file_type), to_categorical(np.array(batch_y),10)

def gen_paths_labels(base_path):
    """A generator to yield (data-paths, corresponding labels) tuples for each
    segment of data (typically training, validation, and testing)."""
    for segment in sorted(os.listdir(base_path)):
        segment_path = os.path.join(base_path, segment)
        segment_paths = []
        segment_labels = []
        for label in os.listdir(segment_path):
            label_path = os.path.join(segment_path, label)
            for crystal in os.listdir(label_path):
                segment_paths.append(os.path.join(label_path, crystal))
                segment_labels.append(label)
        indexes = np.arange(len(segment_labels))
        rng.shuffle(indexes)
        yield [np.array(segment_paths)[indexes], np.array(list(map(int,segment_labels)))[indexes]]

def arrs_from_paths(paths, file_type):
    if file_type == "txt":
        return np.array([np.loadtxt(file_name) for file_name in paths])
    elif file_type == "npy":
        return np.array([np.load(file_name)[[0],:,:] for file_name in paths])

def felix_fit_new(model, batch_size, epochs, workers, AllPaths, file_type, patience):
    #AllPaths = [[TrainingPaths, TrainingThickness], [], []]
    """A fit function to allow validation and test data to be supplied via a
    generator."""
    
    best_val_acc = -np.inf
    
    best_model = None
    
    x = np.arange(0, epochs)
    
    train_seq = FelixSequence(AllPaths[0][0], AllPaths[0][1], batch_size, file_type)
    val_seq = FelixSequence(AllPaths[1][0], AllPaths[1][1], batch_size, file_type)
    test_seq = FelixSequence(AllPaths[2][0], AllPaths[2][1], batch_size, file_type)
    for epoch in range(epochs):
        print("-------------------------------------------------------------------------")
        print("Epoch", epoch+1, "/", epochs, ": ")
        print("Training: ")
        train_hist = model.fit(x = train_seq, epochs = epoch+1, workers = workers, initial_epoch = epoch, shuffle=True)

        print("Validation: ")
        val_hist = model.evaluate(x = val_seq, workers = workers)       
        epoch_acc = val_hist[1]
        if(epoch_acc > best_val_acc):
            best_model = model
            #model.save(NewPath+ModelName)
            print("The model improved from: ",best_val_acc, "to: ", epoch_acc)
            best_val_acc = epoch_acc
            patience_i = 0
        else:
            patience_i+=1
            print("The model did not improve, patience_i = ", patience_i)

        print("Epoch acc: ", epoch_acc)
        #val_hist[0][epoch] = avg_recon_loss
        if(patience_i > patience):
            print("Early Stopping, the model did not improve from: ", best_val_acc)
            break

    print("-------------------------------------------------------------------------")
    print("Testing: ")
    tst_hist = best_model.evaluate(test_seq, workers = workers)
    
    return tst_hist[1], best_model

    

In [17]:
def MakeThicknessList(ListPaths):
    Thickness = []
    for i in ListPaths:
        Thickness.append(int(i.split("/")[-1].split(".")[0]))
    Thickness = np.array(Thickness)
    return(Thickness)

def OpenTxt(Path):
    with open(Path) as textFile:
        lines = [line.split() for line in textFile]
    List = []
    for i in lines:
        List.append(i[0])
    return(List)


DataPath = "//home/ug-ml/felix-ML/classification/Classification000/DataPaths/"

TrainPathAll = OpenTxt(DataPath + "Train_0p1.txt")
ValPathAll = OpenTxt(DataPath + "Validation_0p1.txt")
TestPathAll = OpenTxt(DataPath + "Test_0p1.txt")

In [18]:
LimTrain = int(len(TrainPathAll) * 0.1)
LimVal = int(len(ValPathAll) * 0.1)
LimTest = int(len(TestPathAll) * 0.1)

TrainPath = TrainPathAll[0:LimTrain]
ValPath = ValPathAll[0:LimVal]
TestPath = TestPathAll[0:LimTest]

TrainThickness = MakeThicknessList(TrainPath)[0:LimTrain]
ValThickness = MakeThicknessList(ValPath)[0:LimVal]
TestThickness = MakeThicknessList(TestPath)[0:LimTest]




AllPaths = [[TrainPath,TrainThickness],[ValPath,ValThickness],[TestPath,TestThickness]]

In [19]:
#All paths

SaveDataPath = "/home/ug-ml/Documents/GitHub_BigFiles/SaveFolder" #Base directory of place you store information of models
SaveFolderName = "/Classifer_1" #Will create a folder and put in information about the outcome / inputs
ModelName = "/Model.hdf5"

NewPath = SaveDataPath + SaveFolderName
#Many variables

#Model Variables
input_shape = (1, 128, 128)

#Hyper parameters
learning_rate = 0.0005
l2_regularizer = 0.0001
loss = 'categorical_crossentropy'
optimizer = "RMSprop" #Not a variable ONLY used for a note
batch_size = 32
epochs = 50
ShuffleTrainData = True

#Call back variables
TrainingPatience = 8
CheckPointMonitor = 'val_acc'
EarlyStopMonitor = 'val_acc'

#CPU variables
CPUworkers = 16


#List the name of the variables you want to save in a file
VariableListName = ["input_shape", 
                   "learning_rate", "l2_regularizer", "loss", "optimizer", "batch_size", "epochs", "ShuffleTrainData",
                   "TrainingPatience", "CheckPointMonitor", "EarlyStopMonitor",
                   "CPUworkers"]

#List the variables in the same order as VariableListName
VariableListValues = [input_shape, 
                   learning_rate, l2_regularizer, loss, optimizer, batch_size, epochs, ShuffleTrainData,
                   TrainingPatience, CheckPointMonitor, EarlyStopMonitor,
                   CPUworkers]

In [20]:
def build_model(learning_rate, l2_regularizer, dense_layer_size, kernal_size, kernal_num, opt, drop, loss_func):
    
    dense_layer_size = int(dense_layer_size)
    kernal_size = int(kernal_size)
    #strategy = MirroredStrategy() #Allows multiple GPUs
    
    if loss_func == 0:
        acvt = 'relu'
    elif loss_func == 1:
        acvt = 'sigmoid'
    elif loss_func == 2:
        acvt = 'tanh'
        
    #with strategy.scope():
    model = models.Sequential()
    model.add(layers.Conv2D(kernal_num, (kernal_size, kernal_size),
                                     activation=acvt,
                                     data_format='channels_first',
                                     input_shape= input_shape))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Conv2D(kernal_num, (kernal_size, kernal_size),
                                     data_format='channels_first',
                                     activation=acvt))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Conv2D(kernal_num, (kernal_size, kernal_size),
                                     data_format='channels_first',
                                     activation=acvt))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(drop))
    model.add(layers.Dense(dense_layer_size, activation=acvt,
                           kernel_regularizer = l2(l2_regularizer)))

    model.add(layers.Dense(10, activation='softmax',
                           kernel_regularizer = l2(l2_regularizer)))
    if opt == 0:
        model.compile(loss = loss,
                      optimizer = optimizers.RMSprop(learning_rate = learning_rate),
                      metrics=['acc'])
    elif opt == 1:
        model.compile(loss = loss,
                      optimizer = optimizers.SGD(learning_rate = learning_rate),
                      metrics=['acc'])
    return model

In [22]:
parameters = {"l2_regulariser": np.array([0.00005, 0.000075, 0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0015, 0.002]),
             "batch_size": np.array([8, 12, 16, 24, 32, 48, 64, 86, 128]),
             "dense_layer_size":np.array([32, 48, 64, 86, 128, 186, 256, 336, 512]),
             "kernal_size": np.array([2, 3, 4, 5, 6, 7, 8, 9, 10]),
             "kernal_num": np.array([16, 25, 32, 48, 64, 86, 128, 186, 256]),
             "optimisers": np.array([0,1]),
             "dropout": np.array([0.1, 0.15, 0.2, 0.25, 0.275, 0.3, 0.35, 0.4, 0.45]),
             "loss_func": np.array([0,1,2])}
    
z = np.array([2,2,1,1,3,0,2,0])
prev_searched = np.array([[1,2,1,1,3,0,2,0]])

def neighbours(point, dircs):
    ns = dircs+point
    #print(dircs, point, prev_searched)
    return np.array([i for i in ns if (0<=i).all() and (i<9).all() and (0<=i[5]<2) and (0<=i[7]<3) and not (i == prev_searched).all(axis=1).any()]).astype(int)

def parameter_search(parameters, z, prev_searched, learning_rate):
    
    num_params = len(parameters)
    dircs = np.zeros(shape = (2*num_params, num_params))
    for i, dirc in enumerate(dircs):
        if i < num_params:
            dirc[i] = 1
        else:
            dirc[i-num_params] = -1
    
    best_params = {}
    for key in parameters:
        best_params[key] = np.nan

    best_metrics = {"test_loss": np.inf, "test_acc": -np.inf}

    best_hist = {"train_acc": np.zeros(shape = epochs),
                "train_loss": np.zeros(shape=epochs),
                "val_acc": np.zeros(shape = epochs),
                "val_loss": np.zeros(shape=epochs)}
    
    converged = False
        
    while not converged:
        print("ITERATING OVER:")
        neighs = neighbours(z, dircs)
        num_neighs = neighs.shape[0]
        print(neighs, num_neighs)

        if neighs.size == 0:
            print("No new neighbours available. Saving best model and parameter set so far.")
            #best_model.save(NewPath+ModelName)
            converged = True
            break

        step_params = np.array([parameters[key][neighs[:,j]] for j, key in enumerate(parameters)]).T
        #print(step_params)
                

        converged = True

        for i, param_set in enumerate(step_params):
            print("Parameter set ", i+1," of ", num_neighs,"+++++++++++++++++++++++++++++++++++++++++++++++++++.")

            model = build_model(learning_rate, param_set[0], param_set[2], param_set[3], param_set[4], param_set[5], param_set[6], param_set[7])

            test_acc, best_model = felix_fit_new(model, param_set[1].astype(int), epochs, CPUworkers, AllPaths, "npy", TrainingPatience)

            prev_searched = np.append(prev_searched, neighs[i].reshape(1,num_params), axis=0)

            if test_acc > best_metrics["test_acc"]:
                
                best_metrics["test_acc"] = test_acc
                
                for k, key in enumerate(best_params):
                    best_params[key] = param_set[k]

                z = neighs[i]
                
                #best_model.save(NewPath+ModelName)
                converged = False
        print("best params set:" )
        print(best_params)
        print("best metrics set:" )
        print(best_metrics)
                
parameter_search(parameters, z, prev_searched, learning_rate)

ITERATING OVER:
[[3 2 1 1 3 0 2 0]
 [2 3 1 1 3 0 2 0]
 [2 2 2 1 3 0 2 0]
 [2 2 1 2 3 0 2 0]
 [2 2 1 1 4 0 2 0]
 [2 2 1 1 3 1 2 0]
 [2 2 1 1 3 0 3 0]
 [2 2 1 1 3 0 2 1]
 [2 1 1 1 3 0 2 0]
 [2 2 0 1 3 0 2 0]
 [2 2 1 0 3 0 2 0]
 [2 2 1 1 2 0 2 0]
 [2 2 1 1 3 0 1 0]] 13
Parameter set  1  of  13 +++++++++++++++++++++++++++++++++++++++++++++++++++.
-------------------------------------------------------------------------
Epoch 1 / 50 : 
Training: 
354/354 [==============================] - 8s 23ms/step - loss: 1.7946 - acc: 0.3842
Validation: 
42/42 [==============================] - 1s 20ms/step - loss: 2.5190 - acc: 0.4211
The model improved from:  -inf to:  0.42105263471603394
Epoch acc:  0.42105263471603394
-------------------------------------------------------------------------
Epoch 2 / 50 : 
Training: 
Epoch 2/2
354/354 [==============================] - 8s 22ms/step - loss: 1.4391 - acc: 0.5470
Validation: 
42/42 [==============================] - 1s 22ms/step - loss: 1.9192 - acc: 

Epoch 43/43
354/354 [==============================] - 9s 25ms/step - loss: 0.1790 - acc: 0.9735
Validation: 
42/42 [==============================] - 1s 20ms/step - loss: 3.5731 - acc: 0.7248
The model did not improve, patience_i =  1
Epoch acc:  0.7248120307922363
-------------------------------------------------------------------------
Epoch 44 / 50 : 
Training: 
Epoch 44/44
354/354 [==============================] - 8s 24ms/step - loss: 0.1584 - acc: 0.9763
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 3.7122 - acc: 0.7188
The model did not improve, patience_i =  2
Epoch acc:  0.718796968460083
-------------------------------------------------------------------------
Epoch 45 / 50 : 
Training: 
Epoch 45/45
354/354 [==============================] - 9s 24ms/step - loss: 0.1484 - acc: 0.9783
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 3.3113 - acc: 0.7233
The model did not improve, patience_i =  3
Epoch acc:  0.7233082652091

Epoch 14/14
236/236 [==============================] - 8s 35ms/step - loss: 0.4683 - acc: 0.8515
Validation: 
28/28 [==============================] - 1s 22ms/step - loss: 2.3311 - acc: 0.6466
The model improved from:  0.6375939846038818 to:  0.646616518497467
Epoch acc:  0.646616518497467
-------------------------------------------------------------------------
Epoch 15 / 50 : 
Training: 
Epoch 15/15
236/236 [==============================] - 8s 35ms/step - loss: 0.4179 - acc: 0.8613
Validation: 
28/28 [==============================] - 1s 24ms/step - loss: 2.1814 - acc: 0.6436
The model did not improve, patience_i =  1
Epoch acc:  0.6436090469360352
-------------------------------------------------------------------------
Epoch 16 / 50 : 
Training: 
Epoch 16/16
236/236 [==============================] - 8s 34ms/step - loss: 0.3959 - acc: 0.8743
Validation: 
28/28 [==============================] - 1s 24ms/step - loss: 2.0265 - acc: 0.6797
The model improved from:  0.646616518497467 t

354/354 [==============================] - 8s 24ms/step - loss: 0.7789 - acc: 0.7503
Validation: 
42/42 [==============================] - 1s 17ms/step - loss: 1.7164 - acc: 0.6000
The model improved from:  0.5864661931991577 to:  0.6000000238418579
Epoch acc:  0.6000000238418579
-------------------------------------------------------------------------
Epoch 7 / 50 : 
Training: 
Epoch 7/7
354/354 [==============================] - 8s 24ms/step - loss: 0.6768 - acc: 0.7840: 0s - loss: 0.7
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 2.0568 - acc: 0.6211
The model improved from:  0.6000000238418579 to:  0.621052622795105
Epoch acc:  0.621052622795105
-------------------------------------------------------------------------
Epoch 8 / 50 : 
Training: 
Epoch 8/8
354/354 [==============================] - 8s 24ms/step - loss: 0.6015 - acc: 0.8035
Validation: 
42/42 [==============================] - 1s 20ms/step - loss: 2.3330 - acc: 0.6135
The model did not imp

Epoch 11/11
354/354 [==============================] - 8s 23ms/step - loss: 0.4139 - acc: 0.8678
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 2.3582 - acc: 0.6722
The model improved from:  0.6631578803062439 to:  0.6721804738044739
Epoch acc:  0.6721804738044739
-------------------------------------------------------------------------
Epoch 12 / 50 : 
Training: 
Epoch 12/12
354/354 [==============================] - 8s 23ms/step - loss: 0.3774 - acc: 0.8839
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 2.4540 - acc: 0.6887
The model improved from:  0.6721804738044739 to:  0.688721776008606
Epoch acc:  0.688721776008606
-------------------------------------------------------------------------
Epoch 13 / 50 : 
Training: 
Epoch 13/13
354/354 [==============================] - 8s 24ms/step - loss: 0.3354 - acc: 0.8971
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 2.5547 - acc: 0.6722
The model did not i

...

Epoch 5/5
354/354 [==============================] - 8s 23ms/step - loss: 0.8769 - acc: 0.7186
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 2.1592 - acc: 0.5759
The model improved from:  0.5639097690582275 to:  0.5759398341178894
Epoch acc:  0.5759398341178894
-------------------------------------------------------------------------
Epoch 6 / 50 : 
Training: 
Epoch 6/6
354/354 [==============================] - 8s 24ms/step - loss: 0.7749 - acc: 0.7427
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 1.7202 - acc: 0.5880
The model improved from:  0.5759398341178894 to:  0.5879698991775513
Epoch acc:  0.5879698991775513
-------------------------------------------------------------------------
Epoch 7 / 50 : 
Training: 
Epoch 7/7
354/354 [==============================] - 8s 24ms/step - loss: 0.7036 - acc: 0.7743
Validation: 
42/42 [==============================] - 1s 20ms/step - loss: 2.2248 - acc: 0.6226
The model improved from: 

Epoch 48/48
354/354 [==============================] - 9s 24ms/step - loss: 0.1277 - acc: 0.9772
Validation: 
42/42 [==============================] - 1s 19ms/step - loss: 6.0136 - acc: 0.7113
The model did not improve, patience_i =  2
Epoch acc:  0.7112782001495361
-------------------------------------------------------------------------
Epoch 49 / 50 : 
Training: 
Epoch 49/49
354/354 [==============================] - 8s 24ms/step - loss: 0.1255 - acc: 0.9793
Validation: 
42/42 [==============================] - 1s 20ms/step - loss: 4.9775 - acc: 0.7338
The model did not improve, patience_i =  3
Epoch acc:  0.7338345646858215
-------------------------------------------------------------------------
Epoch 50 / 50 : 
Training: 
Epoch 50/50
354/354 [==============================] - 8s 22ms/step - loss: 0.1222 - acc: 0.9814
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 5.1878 - acc: 0.7323
The model did not improve, patience_i =  4
Epoch acc:  0.732330799102

Epoch 19/19
354/354 [==============================] - 8s 21ms/step - loss: 0.3059 - acc: 0.9122
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 2.7859 - acc: 0.6932
The model improved from:  0.6736842393875122 to:  0.693233072757721
Epoch acc:  0.693233072757721
-------------------------------------------------------------------------
Epoch 20 / 50 : 
Training: 
Epoch 20/20
354/354 [==============================] - 8s 24ms/step - loss: 0.2901 - acc: 0.9162
Validation: 
42/42 [==============================] - 1s 20ms/step - loss: 3.0272 - acc: 0.6962
The model improved from:  0.693233072757721 to:  0.6962406039237976
Epoch acc:  0.6962406039237976
-------------------------------------------------------------------------
Epoch 21 / 50 : 
Training: 
Epoch 21/21
354/354 [==============================] - 7s 21ms/step - loss: 0.2677 - acc: 0.9208
Validation: 
42/42 [==============================] - 1s 20ms/step - loss: 3.3019 - acc: 0.6947
The model did not im

Epoch 41/41
354/354 [==============================] - 8s 24ms/step - loss: 0.1480 - acc: 0.9770- ETA: 2s - loss: 0.1638 - acc:
Validation: 
42/42 [==============================] - 1s 22ms/step - loss: 4.8017 - acc: 0.7173
The model did not improve, patience_i =  1
Epoch acc:  0.7172932624816895
-------------------------------------------------------------------------
Epoch 42 / 50 : 
Training: 
Epoch 42/42
354/354 [==============================] - 9s 25ms/step - loss: 0.1359 - acc: 0.9737
Validation: 
42/42 [==============================] - 1s 20ms/step - loss: 4.7097 - acc: 0.7053
The model did not improve, patience_i =  2
Epoch acc:  0.7052631378173828
-------------------------------------------------------------------------
Epoch 43 / 50 : 
Training: 
Epoch 43/43
354/354 [==============================] - 8s 23ms/step - loss: 0.1309 - acc: 0.9754
Validation: 
42/42 [==============================] - 1s 22ms/step - loss: 5.0929 - acc: 0.7113
The model did not improve, patience_i 

Epoch 34/34
354/354 [==============================] - 8s 23ms/step - loss: 0.1347 - acc: 0.9754: 0s - loss: 0.1
Validation: 
42/42 [==============================] - 1s 17ms/step - loss: 4.2908 - acc: 0.7203
The model did not improve, patience_i =  4
Epoch acc:  0.7203007340431213
-------------------------------------------------------------------------
Epoch 35 / 50 : 
Training: 
Epoch 35/35
354/354 [==============================] - 8s 24ms/step - loss: 0.1239 - acc: 0.9779
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 4.6632 - acc: 0.7098
The model did not improve, patience_i =  5
Epoch acc:  0.7097744345664978
-------------------------------------------------------------------------
Epoch 36 / 50 : 
Training: 
Epoch 36/36
354/354 [==============================] - 8s 23ms/step - loss: 0.1201 - acc: 0.9770
Validation: 
42/42 [==============================] - 1s 21ms/step - loss: 4.7399 - acc: 0.7158
The model did not improve, patience_i =  6
Epoch acc:

In [ ]:
print()

In [110]:
prev_searched = np.array([[0,1,2]])

learning_rate = np.array([0.0005, 0.0001, 0.001, 0.002, 0.005])
l2_regularizer = np.array([0.00005, 0.0001, 0.0005, 0.001, 0.002])
batch_size = np.array([8, 16, 32, 64, 128])

def neighbours(point):
    dircs = np.array([[0,0,1],[0,1,0],[1,0,0],[0,0,-1],[0,-1,0],[-1,0,0]])
    ns = dircs+point
    return np.array([i for i in ns if (0<=i).all() and (i<5).all() and not (i == prev_searched).all(axis=1).any()])


z = np.array([0,1,2])
converged = False
best_test_loss = np.inf
best_lr = np.nan
best_l2_r = np.nan
best_bs = np.nan

while not converged:
    print("ITERATING OVER:")
    neighs = neighbours(z)
    print(neighs)
    #print(neighs)
    if neighs.size == 0:
        print("No new neighbours available. Saving best model and parameter set so far.")
        np.save(NewPath+"/parameter_search.npy", np.array(best_lr,best_l2_r,best_bs))
        best_model.save(NewPath+ModelName)
        converged = True
        break
        

    lr = learning_rate[neighs[:,0]]
    l2_r = l2_regularizer[neighs[:,1]]
    bs = batch_size[neighs[:,2]]

    step_params = np.array([lr, l2_r, bs]).T
    
    converged = True
    
    for i, param_set in enumerate(step_params):
        
        model = build_model(param_set[0], param_set[1])
        
        test_loss, best_model = felix_fit_new(model, param_set[2].astype(int), epochs, CPUworkers, AllPaths, "npy", TrainingPatience)
        
        prev_searched = np.append(prev_searched, neighs[i].reshape(1,3), axis=0)
        
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_lr = param_set[0]
            best_l2_r = param_set[1]
            best_bs = param_set[2]
            
            z = neighs[i]
            
            np.save(NewPath+"/parameter_search.npy",param_set)
            best_model.save(NewPath+ModelName)
            converged = False
            
    

ITERATING OVER:
[[0 1 3]
 [0 2 2]
 [1 1 2]
 [0 1 1]
 [0 0 2]]
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
-------------------------------------------------------------------------
Epoch 1 / 1 : 
Training: 
89/89 [==============================] - 7s 83ms/step - loss: 2.0421 - acc: 0.2734
Validation: 
11/11 [==============================] - 0s 29ms/step - loss: 2.5310 - acc: 0.3549
The model improved from:  inf to:  2.5309948921203613
Epoch loss:  2.5309948921203613
-------------------------------------------------------------------------
Testing: 
6/6 [==============================] - 0s 20ms/step - loss: 1.9448 - acc: 0.3724
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
-------------------------------------------------------------------------
Epoch 1 / 1 : 
Training: 
177/177 [=

In [112]:
print(best_lr,best_l2_r,best_bs)
print(best_test_loss)

0.0005 0.0005 32.0
1.822420358657837


In [38]:
test_seq = FelixSequence(AllPaths[2][0], AllPaths[2][1], best_bs.astype(int), "npy")
tst_hist = best_model.evaluate(test_seq, workers = 16)

11/11 [==============================] - 0s 10ms/step - loss: 1.6834 - acc: 0.6216


In [ ]:
#Build model
strategy = MirroredStrategy() #Allows multiple GPUs

with strategy.scope():
    model = models.Sequential()
    model.add(layers.Conv2D(128, (4, 4),
                                     activation='relu',
                                     data_format='channels_first',
                                     input_shape= input_shape))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Conv2D(128, (4, 4),
                                     data_format='channels_first',
                                     activation='relu'))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Conv2D(128, (4, 4),
                                     data_format='channels_first',
                                     activation='relu'))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(128, activation='relu',
                           kernel_regularizer = l2(l2_regularizer)))
    
    model.add(layers.Dense(10, activation='softmax',
                           kernel_regularizer = l2(l2_regularizer)))

    model.compile(loss = loss,
                  optimizer = optimizers.RMSprop(learning_rate = learning_rate),
                  metrics=['acc'])
    
#Make folder to put model and history information
try:
    os.mkdir(NewPath)
except:
    print("Folder failed to be created, it may already exist")
    
File1  = open(NewPath +"/Parameters.txt", "w+")
if(len(VariableListName) == len(VariableListValues)):
    for i in range(0, len(VariableListName)):
        File1.write(VariableListName[i] + " " + str(VariableListValues[i]) + "\n")
    File1.close()
else:
    print("VariableListName and VariableListValues do not match up, so file can not be saved")

In [ ]:
training_history = np.zeros(shape=(2,epochs))
validation_history = np.zeros(shape=(2,epochs))
test_history = [0,0]
#print(model.metrics_names)
felix_fit_new(model, batch_size, epochs, CPUworkers, AllPaths, "npy",TrainingPatience)